In [10]:
import numpy as np
import pandas as pd
from math import radians, cos, sin, sqrt, atan2

# 거리 계산 함수 (Haversine 공식)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2) * sin(dlat/2) + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2) * sin(dlon/2)
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance
df_districts = pd.read_excel('달서구 행정구역 중심좌표.xlsx')
df_hospitals = pd.read_csv('대구광역시 달서구_의료기관 현황_20240520.csv',encoding='cp949')
df_hospitals['위도'] = df_hospitals['위도'].astype(str)
df_hospitals['경도'] = df_hospitals['경도'].astype(str)

# Extract numeric values and convert to float
df_hospitals['위도'] = df_hospitals['위도'].str.extract(r'([-+]?\d*\.\d+|\d+)').astype(float)
df_hospitals['경도'] = df_hospitals['경도'].str.extract(r'([-+]?\d*\.\d+|\d+)').astype(float)
def calculate_infrastructure_score(lat, lon, hospitals, radius=0.5):
    scores = []
    for _, row in hospitals.iterrows():
        dist = haversine(lat, lon, row['위도'], row['경도'])
        if dist <= radius:
            score = 1 / (dist + 0.1)  # 역수 방식의 점수, 0으로 값이 나오는 것을 방지하기 위해 0.1 추가
            scores.append(score)
    return sum(scores)  # 또는 np.mean(scores)로 평균 점수

# 각 행정동에 대해 인프라 점수 계산
df_districts['인프라점수'] = df_districts.apply(
    lambda row: calculate_infrastructure_score(row['위도'], row['경도'], df_hospitals),
    axis=1
)

print(df_districts[['행정구역', '인프라점수']])

   행정구역       인프라점수
0   성당동    0.000000
1   두류동   90.839871
2   본리동   29.569394
3   감삼동  107.023547
4   죽전동   61.620683
5   장기동   13.757485
6   용산동   85.428625
7   이곡동   86.488309
8   신당동    9.231426
9   월성동   57.444971
10  진천동  164.957285
11  유천동   30.813871
12  상인동   61.973336
13  도원동    3.431355
14  송현동   90.147138
15   본동   24.183295


In [11]:
df_districts.to_excel('달서구 행정구역 인프라 점수.xlsx', index=False)